# Links

In [27]:
import pandas as pd
import os
from utils.youtube_extractor import extrair_links_playlist  # Ajuste conforme o caminho do seu módulo

# URL da playlist do YouTube
url_playlist = "https://www.youtube.com/playlist?list=PLcwPvhFHJ_F83aRklVj8vc6f4zIZafMKn"

# Obter nome da playlist e informações dos vídeos
nome_playlist, videos_info = extrair_links_playlist(url_playlist)

# Converter a lista de dicionários em DataFrame
videos_df = pd.DataFrame(videos_info)

# Exibindo os resultados
print("Nome da Playlist:", nome_playlist)
videos_df.head()

100%|██████████| 20/20 [00:26<00:00,  1.31s/it]

Nome da Playlist: Itaú Views | 5ª Temporada


,titulo,url,duração,visualizações,avaliação,autor,descrição,data_publicação,curtidas,keywords,número_curtidas,número_descurtidas
0,Privatização da Sabesp: o que esperar pela fre...,https://youtube.com/watch?v=nlpEgrjDC-8,1941,356,None,Itaú Views | 5ª Temporada,None,2023-12-18,None,[],None,None
1,Qual o melhor trade com renda fixa?,https://youtube.com/watch?v=8nxRcMYud8g,1769,621,None,Itaú Views | 5ª Temporada,None,2023-12-11,None,[],None,None
2,Um raio-X da China,https://youtube.com/watch?v=Wp5mBfznTyk,1299,402,None,Itaú Views | 5ª Temporada,None,2023-12-04,None,[],None,None
3,AI e a corrida da tecnologia,https://youtube.com/watch?v=Kr9z9GofAZw,2040,246,None,Itaú Views | 5ª Temporada,None,2023-11-27,None,[],None,None
4,Ibovespa em 145 mil pontos,https://youtube.com/watch?v=bq5Q_Fjd8Ms,1653,617,None,Itaú Views | 5ª Temporada,None,2023-11-20,None,[],None,None


In [33]:
# Salvar links
videos_df.to_csv(os.path.join("../data/links/",f"links_{nome_playlist}.csv"), index=False)

# Video

In [1]:
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import OpenAIWhisperParser
from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader
url="https://www.youtube.com/watch?v=my-79yTtI5s"
save_dir="docs/youtube/"
loader = GenericLoader(
    YoutubeAudioLoader([url],save_dir),
    OpenAIWhisperParser()
)
docs = loader.load()

[youtube] Extracting URL: https://www.youtube.com/watch?v=my-79yTtI5s
[youtube] my-79yTtI5s: Downloading webpage
[youtube] my-79yTtI5s: Downloading ios player API JSON
[youtube] my-79yTtI5s: Downloading android player API JSON


[youtube] my-79yTtI5s: Downloading player 8fc6998a
[youtube] my-79yTtI5s: Downloading m3u8 information
[info] my-79yTtI5s: Downloading 1 format(s): 140
[download] Destination: docs/youtube//El Niño： Impactos e Oportunidades para os investidores.m4a
[download] 100% of   11.69MiB in 00:00:05 at 2.20MiB/s     
[FixupM4a] Correcting container of "docs/youtube//El Niño： Impactos e Oportunidades para os investidores.m4a"
[ExtractAudio] Not converting audio docs/youtube//El Niño： Impactos e Oportunidades para os investidores.m4a; file is already in target format m4a
Transcribing part 1!


In [2]:
docs[0].page_content[0:500]

'Oi, pessoal, aqui quem fala é Gabriela Bonzodas e esse é mais um episódio do podcast Estão Ouvidos. Antes de começar, queria lembrar vocês que os nossos episódios estão disponíveis também em vídeo no YouTube e que nós estamos marcando presença no Instagram e no Telegram, onde vocês encontram análises exclusivas. Essa semana nós vamos falar dos efeitos que o fenômeno climático El Niño tem em determinadas culturas aqui do agronegócio e como isso pode se desdobrar em impactos para as empresas da no'

In [3]:
from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader

In [4]:
# from langchain_community.document_loaders.blob_loaders.youtube_audio import (
#     YoutubeAudioLoader,
# )
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import (
    OpenAIWhisperParser,
)
from langchain.document_loaders.parsers.audio import OpenAIWhisperParserLocal

In [5]:
# set a flag to switch between local and remote parsing
# change this to True if you want to use local parsing
local = True

In [13]:
import pandas as pd
videos_df = pd.read_csv("../data/links/links_Itaú Views | 5ª Temporada.csv")
url_menor_video = videos_df[videos_df['duração'] == videos_df['duração'].min()].url.values[0]

In [51]:
from transformers import WhisperProcessor
processor = WhisperProcessor.from_pretrained("openai/whisper-small")
processor.get_decoder_prompt_ids(language="portuguese", task="translate")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[(1, 50267), (2, 50358), (3, 50363)]

In [52]:
# Two Karpathy lecture videos
urls = videos_df.url.values.tolist() #[url_menor_video]

# Directory to save audio files
save_dir = "../data/videos/"


# Transcribe the videos to text
if local:
    loader = GenericLoader(
        # exemplo "../data/videos/Balanço dos Balanços 2T23.m4a!" necessario passar a lingua por conta do erro: Multiple languages detected when trying to predict the most likely target language for transcription. It is currently not supported to transcribe to different languages in a single batch. Please make sure to either force a single language by passing `language='...'` or make sure all input audio is of the same language.
        YoutubeAudioLoader(urls, save_dir), OpenAIWhisperParserLocal(chunk_length=20, # quando chunk=30, houve alguns bugs, palavras repetidas na transcrição
                                                                     forced_decoder_ids=processor.get_decoder_prompt_ids(language="portuguese", task="translate")
                                                                     )
        
    )
else:
    loader = GenericLoader(YoutubeAudioLoader(urls, save_dir), OpenAIWhisperParser())
docs = loader.load()

Using the following model:  openai/whisper-small


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[youtube] Extracting URL: https://youtube.com/watch?v=nlpEgrjDC-8
[youtube] nlpEgrjDC-8: Downloading webpage
[youtube] nlpEgrjDC-8: Downloading ios player API JSON
[youtube] nlpEgrjDC-8: Downloading android player API JSON


[youtube] nlpEgrjDC-8: Downloading m3u8 information
[info] nlpEgrjDC-8: Downloading 1 format(s): 140
[download] ../data/videos//Privatização da Sabesp： o que esperar pela frente？.m4a has already been downloaded
[download] 100% of   29.94MiB
[ExtractAudio] Not converting audio ../data/videos//Privatização da Sabesp： o que esperar pela frente？.m4a; file is already in target format m4a
[youtube] Extracting URL: https://youtube.com/watch?v=8nxRcMYud8g
[youtube] 8nxRcMYud8g: Downloading webpage
[youtube] 8nxRcMYud8g: Downloading ios player API JSON
[youtube] 8nxRcMYud8g: Downloading android player API JSON


[youtube] 8nxRcMYud8g: Downloading m3u8 information
[info] 8nxRcMYud8g: Downloading 1 format(s): 140
[download] ../data/videos//Qual o melhor trade com renda fixa？.m4a has already been downloaded
[download] 100% of   27.29MiB
[ExtractAudio] Not converting audio ../data/videos//Qual o melhor trade com renda fixa？.m4a; file is already in target format m4a
[youtube] Extracting URL: https://youtube.com/watch?v=Wp5mBfznTyk
[youtube] Wp5mBfznTyk: Downloading webpage
[youtube] Wp5mBfznTyk: Downloading ios player API JSON
[youtube] Wp5mBfznTyk: Downloading android player API JSON


...

[youtube] W1rxLjimvxY: Downloading m3u8 information
[info] W1rxLjimvxY: Downloading 1 format(s): 140
[download] ../data/videos//Buy Side & Sell Side ｜ Fundos Imobiliários.m4a has already been downloaded
[download] 100% of   23.30MiB
[ExtractAudio] Not converting audio ../data/videos//Buy Side & Sell Side ｜ Fundos Imobiliários.m4a; file is already in target format m4a
[youtube] Extracting URL: https://youtube.com/watch?v=yRJ5j1nS6SM
[youtube] yRJ5j1nS6SM: Downloading webpage
[youtube] yRJ5j1nS6SM: Downloading ios player API JSON
[youtube] yRJ5j1nS6SM: Downloading android player API JSON


[youtube] yRJ5j1nS6SM: Downloading m3u8 information
[info] yRJ5j1nS6SM: Downloading 1 format(s): 140
[download] ../data/videos//Juros pra baixo, bolsa pra... baixo？.m4a has already been downloaded
[download] 100% of   28.70MiB
[ExtractAudio] Not converting audio ../data/videos//Juros pra baixo, bolsa pra... baixo？.m4a; file is already in target format m4a
[youtube] Extracting URL: https://youtube.com/watch?v=3vUnTO2Gmsg
[youtube] 3vUnTO2Gmsg: Downloading webpage
[youtube] 3vUnTO2Gmsg: Downloading ios player API JSON
[youtube] 3vUnTO2Gmsg: Downloading android player API JSON


[youtube] 3vUnTO2Gmsg: Downloading m3u8 information
[info] 3vUnTO2Gmsg: Downloading 1 format(s): 140
[download] ../data/videos//Orange Book： um raio-x da nossa economia.m4a has already been downloaded
[download] 100% of   25.78MiB
[ExtractAudio] Not converting audio ../data/videos//Orange Book： um raio-x da nossa economia.m4a; file is already in target format m4a
[youtube] Extracting URL: https://youtube.com/watch?v=y3aSGWTeypc
[youtube] y3aSGWTeypc: Downloading webpage
[youtube] y3aSGWTeypc: Downloading ios player API JSON
[youtube] y3aSGWTeypc: Downloading android player API JSON


[youtube] y3aSGWTeypc: Downloading m3u8 information
[info] y3aSGWTeypc: Downloading 1 format(s): 140
[download] ../data/videos//O que esperar da política monetária no Brasil？.m4a has already been downloaded
[download] 100% of   19.86MiB
[ExtractAudio] Not converting audio ../data/videos//O que esperar da política monetária no Brasil？.m4a; file is already in target format m4a
[youtube] Extracting URL: https://youtube.com/watch?v=plW9ZiMFcOM
[youtube] plW9ZiMFcOM: Downloading webpage
[youtube] plW9ZiMFcOM: Downloading ios player API JSON
[youtube] plW9ZiMFcOM: Downloading android player API JSON


[youtube] plW9ZiMFcOM: Downloading m3u8 information
[info] plW9ZiMFcOM: Downloading 1 format(s): 140
[download] ../data/videos//Balanço dos Balanços 2T23.m4a has already been downloaded
[download] 100% of   28.54MiB
[ExtractAudio] Not converting audio ../data/videos//Balanço dos Balanços 2T23.m4a; file is already in target format m4a
Transcribing part ../data/videos/AI e a corrida da tecnologia.m4a!


In [ ]:
# chat https://python.langchain.com/docs/integrations/document_loaders/youtube_audio/